# Codice per la generazione di grafi casuali
## Strumento di supporto per il prog. di Programmazione su Arch. Parallele

*Lena Emanuele, Zuccato Francesco*

In [ ]:
import networkx as nx
import numpy as np
import pandas as pd

## Creazione grafi casuali
Generazione di grafi casuali con specifiche caratteristiche

In [ ]:
n = 1000     # numero di nodi
seed = 1234   # seed per replicare i risultati

In [ ]:
# Grafo casuale semplice

p = 0.1 # probabilità per ogni coppia di nodi di creare un arco
g = nx.erdos_renyi_graph(n, p, seed=seed, directed=False)

In [ ]:
# Grafo piccolo-mondo (sostanzialmente, un grafo casuale dove 
# la distanza media tra due nodi è bassa). Si genera:
#   - connettendo in modo "geometrico" ogni nodo a quelli che sarebbero i
#     k nodi più vicini in una rete ad anello
#   - per ogni arco, con una prob. p viene scollegato e ri-collegato
#     ad un nodo casuale
# 
# Ciò che ne esce è un grafo semi-regolare ma con anche connessioni casuali

k=5
p=0.1
g = nx.connected_watts_strogatz_graph(n, k, p, seed=seed)

# (Varianti)
# g = nx.watts_strogatz_graph(n, k, p, seed=seed) # non garantisce sia connesso
# g = nx.newman_watts_strogatz_graph(n, k, p, seed=seed) # per generare il collegamento random non sostituisce il precedente

In [ ]:
# Grafo con distribuzione dei gradi in power law:
# sostanzialmente, un grafo dove ci saranno pochi nodi con grado molto alto 
# (quindi pochi nodi con molte connessioni, e tante con poche connessioni)

# [...] (è complicato da spiegare... https://networkx.org/documentation/stable/reference/generated/networkx.generators.random_graphs.barabasi_albert_graph.html)
m = 25  
g = nx.barabasi_albert_graph(n, m, seed=seed)

In [ ]:
g = nx.Graph()



### Generazione di pesi casuali
I grafi generati in questo modo NON sono pesati! Gli assegnamo quindi dei pesi casuali



In [ ]:
def attachRandomWeight(g: nx.Graph, minWeight: int, maxWeight: int, seed: int) -> nx.Graph:
  """
    funzione che dato un grafo e un intervallo di valori, assegna
    (pseudo) casualmente dei pesi a ciascun arco. 

    Si ritorna un nuovo grafo con i pesi.
  """

  gg = g.copy()

  np.random.seed(seed)
  weights = np.random.randint(1, 10, len(gg.edges))

  i = 0
  for (u,v) in gg.edges():
    gg.edges[u,v]['weight'] = weights[i]
    i += 1

  return gg

In [ ]:
g = attachRandomWeight(g, 1, 5, seed)

### Controllo connessione
Controlliamo che il grafo sia connesso

In [ ]:
nx.is_connected(g)

True

## Esportazione

### Estrazione della matrice di adiacenza

In [ ]:
def getAdjMatrix(g: nx.Graph, noEdgeValue: int = -1) -> np.matrix:
  """
  Genera la matrice di adiacenza da un grafo, 
  riempiendo come da indicazione i valori nulli
  """

  # esportazione matrice in forma densa
  adj_matrix = nx.adjacency_matrix(g).todense()

  # inserimento del valore noEdgeValue dove non ci sono archi
  for i in range(adj_matrix.shape[0]):
    for j in range(adj_matrix.shape[1]):
      if adj_matrix[i, j] == 0 and i!=j and not g.has_edge(i, j):
        adj_matrix[i, j] = noEdgeValue

  return adj_matrix


In [ ]:
MAX_INT = 2147483647
adj_matrix = getAdjMatrix(g, noEdgeValue = MAX_INT)

In [ ]:
adj_matrix

matrix([[         0, 2147483647, 2147483647, ..., 2147483647,          1,
         2147483647],
        [2147483647,          0, 2147483647, ..., 2147483647, 2147483647,
                  9],
        [2147483647, 2147483647,          0, ..., 2147483647,          2,
         2147483647],
        ...,
        [2147483647, 2147483647, 2147483647, ...,          0, 2147483647,
                  3],
        [         1, 2147483647,          2, ..., 2147483647,          0,
         2147483647],
        [2147483647,          9, 2147483647, ...,          3, 2147483647,
                  0]], dtype=int64)

### Download in formato CSV

In [ ]:
def downloadAdjMatrixAsCsv(adjMatrix: np.matrix, filename:str="graph_adj_matrix.csv", colabDownload=False) -> None:
  """
    scarica una matrice di adiacenza come CSV. 

    Si omettono sia i nomi delle righe che delle colonne.
  """

  df = pd.DataFrame(data=adjMatrix.astype(int))
  df.to_csv(filename, sep=' ', header=False, index=False)

  if colabDownload:

    from google.colab import files
    files.download(filename) 



In [ ]:
downloadAdjMatrixAsCsv(adj_matrix)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>